In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import csv

In [ ]:
pythia_fragment_CP5 = """
from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8ConcurrentHadronizerFilter",
                         maxEventsToPrint = cms.untracked.int32(1),
                         pythiaPylistVerbosity = cms.untracked.int32(1),
                         filterEfficiency = cms.untracked.double(1.0),
                         pythiaHepMCVerbosity = cms.untracked.bool(False),
                         comEnergy = cms.double(13000.),
                         PythiaParameters = cms.PSet(
                                                     pythia8CommonSettingsBlock,
                                                     pythia8CP5SettingsBlock,
                                                     pythia8PSweightsSettingsBlock,
                                                     parameterSets = cms.vstring('pythia8CommonSettings',
                                                                                 'pythia8CP5Settings',
                                                                                 'pythia8PSweightsSettings',
                                                                                 )
                                                     )
                         )


ProductionFilterSequence = cms.Sequence(generator)
"""

In [ ]:
def fragmentsDictCreator():
    dict = { '2016': pythia_fragment_CP5,
             '2016APV': pythia_fragment_CP5,
             '2017': pythia_fragment_CP5,
             '2018': pythia_fragment_CP5
                    }
    return dict

In [ ]:
LHEproducer = """
import FWCore.ParameterSet.Config as cms

# link to cards:
# __EXAMPLE__

externalLHEProducer = cms.EDProducer('ExternalLHEProducer',
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    generateConcurrently = cms.untracked.bool(False),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)
"""

In [ ]:
years = ["2016", "2016APV", "2017", "2018"]
mX = ["500", "750", "1000", "1250", "1500", "1750", "2000", "2500", "3000", "4000"]
scenario = ["4", "10", "20", "50"]
tot_events1 = [1000000]*len(mX)
gridpacks_dict = {}
example_dict = {}
dataset_names = {}

In [ ]:
for year in range(len(years)):
    t_example1 = 'https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/TopPhilicZp/ttZp'
    t_datasetname_year = 'TTZprimeToTT_M-{massX}_Width{scenario}_TuneCP5_13TeV-madgraph-pythia8'
    t_gp_year = "/cvmfs/cms.cern.ch/phys_generator/gridpacks/UL/13TeV/slc7_amd64_gcc700/MadGraph5_aMCatNLO/TTZprimeToTT/TopPhilic_ttzp_m{massX}_relwidth{scenario}_madspin_tarball.tar.xz"    
    tmp_dataset_dict={}
    tmp_gridpack_dict={}
    tmp_example_dict={}
    dataset_names_year = []
    gp_ggf_year = "gp_ggf_" + years[year]
    gp_ggf_year = []
    
    dataset_names_year = []
    gp_ggf_year = []

    for mx in mX:
        for scena in scenario:
            d = {"massX": mx, "scenario": scena}
            tmp_example_dict = t_example1
            gp_ggf_year.append(t_gp_year.format_map(d))
            dataset_names_year.append(t_datasetname_year.format_map(d))
            tmp_dataset_dict = dataset_names_year
            tmp_gridpack_dict = gp_ggf_year
        
    gridpacks_dict[years[year]] = tmp_gridpack_dict
    dataset_names[years[year]] = tmp_dataset_dict
    example_dict[years[year]] = tmp_example_dict

## .csv Production

In [ ]:
#MadgraphVersion = re.compile("V5_2\.[0-9]\.[0-9]")
process_pythia_map = {}
tmp_process_pythia_map = {}
tmp_process_pythia_map = fragmentsDictCreator()
process_pythia_map.update(tmp_process_pythia_map)

In [ ]:
#for year in range(len(years)):
for year in gridpacks_dict:
    #print(year)
    with open('ttzprimetott' + '_' + year +'_modify.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',',
                        quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(['prepid','Dataset name','Events', 'fragment','notes','Generator','mcdbid','time','size'])
        if year == "2016":
            pr = 5208
        if year == "2016APV":
            pr = 5224
        if year == "2017":
            pr = 4824
        if year == "2018":
            pr = 5152
            
        k=0
        for i in range(len(mX)):
            for j in range(len(scenario)):
                tmp_fragment=""
                #print(gridpacks_dict[year][i])
                #print(dataset_names[year][i])
                #version = MadgraphVersion.search(path).group(0)
                version="2.6.5"
                dataset_name = dataset_names[year][k]
                if year == "2016APV":
                    events = round(tot_events1[i]*0.54)
                    prepid = 'B2G-RunIISummer20UL16wmLHEGENAPV-0' + str(pr)
                elif year == "2016":
                    events = round(tot_events1[i]*0.46)
                    prepid = 'B2G-RunIISummer20UL16wmLHEGEN-0' + str(pr)
                elif year == "2017":
                    events = tot_events1[i]
                    prepid = 'B2G-RunIISummer20UL17wmLHEGEN-0' + str(pr)
                elif year == "2018":
                    events = tot_events1[i]
                    prepid = 'B2G-RunIISummer20UL18wmLHEGEN-0' + str(pr)
                
                tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][k]) + '\n' + process_pythia_map[year]
                final_fragment = tmp_fragment.replace('__EXAMPLE__',example_dict[year])
                note = dataset_name.replace('_',' ')
                generators="Madgraph_" + version + "  Madspin Pythia8"
                mcdb_id = '0'
                time = '0.01'
                size = '30'
                csvwriter.writerow([prepid, dataset_name, events, final_fragment, note, generators, mcdb_id, time, size])
                pr = pr+1   
                k=k+1